In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r"C:\Users\22303\Desktop\6013小组作业\dataset.csv")

In [3]:
from imblearn.over_sampling import SMOTE

# 分离特征（X）和标签（y）
X = df.drop("loan_status", axis=1)  # 特征列
y = df["loan_status"]  # 标签列

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=114514, stratify=y)

# 初始化SMOTE过采样器
smote = SMOTE(random_state=114514)
X_train, y_train = smote.fit_resample(X_train, y_train)

# 5. 查看采样后的数据分布
print("\n采样后训练集标签分布：")
print(y_train.value_counts())

# 合并训练集特征（X_train）和标签（y_train）
train_set = pd.concat([X_train, y_train], axis=1)

# 合并测试集特征（X_test）和标签（y_test）
test_set = pd.concat([X_test, y_test], axis=1)


采样后训练集标签分布：
loan_status
0.0    1570803
1.0    1570803
Name: count, dtype: int64


In [4]:
# 公式：标签 ~ 特征1 + 特征2 + 特征3，C()表示分类变量
# 公式中各变量含义说明：
# loan_status：因变量，代表贷款状态（如是否违约等二分类标签）
# sub_grade：贷款子等级，是对贷款风险的细分评级
# dti：债务收入比，借款人债务总额与年收入的比例，用于评估还款能力
# annual_inc：借款人年收入，体现经济实力与还款能力
# fico：FICO信用分，国际通用的个人信用风险评估指标
# emp_length：就业时长，反映借款人收入的稳定性
# int_rate：贷款年化利率，与贷款风险和资金成本直接相关
# term：贷款期限（如36个月、60个月），影响还款周期与压力
# delinq_2yrs：近两年逾期次数，体现借款人的信用历史表现
# pub_rec：公共记录不良次数（如破产、法律判决等），反映严重信用风险
formula = '''loan_status ~ sub_grade + dti + annual_inc + fico 
           + emp_length + int_rate + term + delinq_2yrs + pub_rec'''

# 初始化模型
model = Logit.from_formula(formula, data=train_set)

# 拟合模型
results = model.fit()

Optimization terminated successfully.
         Current function value: 0.624171
         Iterations 5


In [5]:
# 预测正类（1）的概率
y_pred_prob = results.predict(test_set)

# 概率转类别（阈值0.5）
y_pred = (y_pred_prob > 0.5).astype(int)

# 从测试集中提取标签
y_test = test_set['loan_status']

# 混淆矩阵（True Negative, False Positive, False Negative, True Positive）
print("混淆矩阵：")
print(confusion_matrix(y_test, y_pred))

# 分类报告（准确率、召回率、F1-score）
print("\n分类报告：")
print(classification_report(y_test, y_pred))

# AUC值（评估模型区分能力，0.5=随机，1=完美）
auc = roc_auc_score(y_test, y_pred_prob)
print(f"\nAUC值：{auc:.4f}")

混淆矩阵：
[[253203 139498]
 [ 20258  37755]]

分类报告：
              precision    recall  f1-score   support

         0.0       0.93      0.64      0.76    392701
         1.0       0.21      0.65      0.32     58013

    accuracy                           0.65    450714
   macro avg       0.57      0.65      0.54    450714
weighted avg       0.83      0.65      0.70    450714


AUC值：0.7056


In [6]:
# 模型指标与实验结果评估
# 本次逻辑回归模型针对贷款状态预测任务展开，
# 结合数据处理流程与评估指标，整体表现中等偏上，
# 具体分析如下：

# 1. 数据处理效果
# 实验采用SMOTE算法对训练集进行过采样，
# 最终使正负样本（loan_status=0.0和1.0）数量均达到1570803，
# 有效解决了原始数据的类别不平衡问题，
# 为模型训练提供了均衡的样本基础。

# 2. 核心指标表现
# - **AUC值**：0.7056，处于中等偏上水平，
#   说明模型对正负样本的区分能力一般，
#   具备一定的预测辨识度，但未达到优秀标准（通常AUC≥0.8为优秀）。
# - **整体准确率**：0.65，
#   反映模型在测试集（450714个样本）上的整体预测正确性，
#   表现平平，存在较大提升空间。
# - **类别级指标差异**：
#   - 负类（loan_status=0.0，推测为正常还款）：
#     精确率高达0.93，说明预测为正常还款的样本中93%为真实正例，
#     但召回率仅0.64，意味着36%的真实正常还款样本被误判为违约，
#     存在较多漏判。
#   - 正类（loan_status=1.0，推测为违约还款）：
#     召回率0.65，能够捕捉到65%的真实违约样本，
#     满足“尽可能识别风险”的核心需求，
#     但精确率仅0.21，即预测为违约的样本中仅21%为真实违约，
#     大量正常样本被误判，假阳性率过高。

# 3. 实验结果总结
# - 优点：
#   SMOTE过采样有效避免了模型偏向多数类，
#   正类召回率达标，
#   可初步满足风险识别场景中“减少漏判风险”的基础要求；
#   模型训练收敛顺利（迭代5次完成优化），
#   计算效率较高。
# - 不足：
#   正类精确率极低，会导致大量误判成本（如误将正常用户判定为高风险）；
#   整体准确率和AUC值偏低，
#   模型的综合预测能力有限。